# 02 - LPPLS Bubble Detection

The **Log-Periodic Power Law Singularity (LPPLS)** model detects financial bubbles by
fitting a characteristic oscillatory pattern to log-prices. As a market approaches a
critical point (crash), the oscillation frequency increases -- a signature of
positive feedback loops among traders.

This notebook:
1. Loads BTC price data
2. Fits the LPPLS model to historical windows
3. Computes the LPPLS confidence indicator
4. Visualizes detected bubble periods against actual price history

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from fatcrash.data.ingest import from_yahoo
from fatcrash.data.transforms import log_prices, time_index
from fatcrash.data.cache import load_cached, save_cache
from fatcrash.indicators.lppls_indicator import fit_lppls, compute_confidence
from fatcrash.viz.charts import plot_price_with_confidence

plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (14, 6)

## 1. Load data

In [ ]:
# Load BTC daily data
df = from_yahoo("BTC-USD", start="2015-01-01", end="2025-12-31")
df = time_index(df)

# Compute log prices for LPPLS fitting
prices = df["close"].values
log_p = log_prices(prices)
dates = df.index

print(f"Observations: {len(df)}")
print(f"Date range: {dates[0].date()} to {dates[-1].date()}")

## 2. Single LPPLS fit

We first demonstrate a single LPPLS fit on the 2017 bubble period.
The LPPLS model has the form:

$$\ln p(t) = A + B|t_c - t|^m + C|t_c - t|^m \cos(\omega \ln|t_c - t| + \phi)$$

where $t_c$ is the estimated critical time (crash date).

In [ ]:
# Focus on the 2017 bubble (roughly Jan 2017 - Dec 2017)
mask_2017 = (dates >= "2017-01-01") & (dates <= "2017-12-17")
t_2017 = np.arange(mask_2017.sum(), dtype=np.float64)
lp_2017 = log_p[mask_2017]

# Fit LPPLS
result = fit_lppls(t_2017, lp_2017)

print("LPPLS fit parameters:")
print(f"  tc (critical time index): {result.tc:.1f}")
print(f"  m  (power exponent):      {result.m:.4f}")
print(f"  omega (log-frequency):    {result.omega:.4f}")
print(f"  A:                        {result.A:.4f}")
print(f"  B:                        {result.B:.6f}")
print(f"  C:                        {result.C:.6f}")

In [ ]:
# Plot the fit
fig, ax = plt.subplots(figsize=(14, 6))

dates_2017 = dates[mask_2017]
ax.plot(dates_2017, lp_2017, "k-", linewidth=0.8, label="Log Price (actual)")

# Reconstruct fitted values
dt = np.abs(result.tc - t_2017)
dt = np.maximum(dt, 1e-10)  # avoid log(0)
fitted = result.A + result.B * dt**result.m + result.C * dt**result.m * np.cos(
    result.omega * np.log(dt) + result.phi
)
ax.plot(dates_2017, fitted, "r-", linewidth=1.2, alpha=0.8, label="LPPLS Fit")

# Mark estimated critical time
if result.tc < len(dates_2017) + 60:
    tc_date = dates_2017[0] + pd.Timedelta(days=int(result.tc))
    ax.axvline(tc_date, color="red", linestyle="--", alpha=0.6, label=f"tc = {tc_date.date()}")

ax.set_xlabel("Date")
ax.set_ylabel("Log Price")
ax.set_title("LPPLS Fit: 2017 BTC Bubble")
ax.legend()
plt.tight_layout()
plt.show()

## 3. Compute LPPLS confidence indicator

The confidence indicator runs LPPLS fits across multiple windows and parameter
filters. A high confidence value means many independent fits agree that a bubble
is in progress.

In [ ]:
# Compute rolling LPPLS confidence over the full series
# This uses multiple window sizes and filters valid fits
t_full = np.arange(len(log_p), dtype=np.float64)

confidence = compute_confidence(
    t_full,
    log_p,
    window_sizes=[120, 180, 250, 365],  # multiple windows in days
    step=5,  # compute every 5 days for speed
)

# Build a DataFrame with the results
conf_df = pd.DataFrame({
    "date": dates,
    "close": prices,
    "positive_confidence": confidence.positive,
    "negative_confidence": confidence.negative,
}).set_index("date")

print(f"Max positive confidence: {conf_df['positive_confidence'].max():.3f}")
print(f"Max negative confidence: {conf_df['negative_confidence'].max():.3f}")

## 4. Visualize bubble periods

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Price (log scale)
axes[0].plot(conf_df.index, conf_df["close"], color="steelblue", linewidth=0.8)
axes[0].set_yscale("log")
axes[0].set_ylabel("Price (USD)")
axes[0].set_title("BTC/USD Price with LPPLS Confidence")

# Positive confidence (bubble)
axes[1].fill_between(
    conf_df.index, 0, conf_df["positive_confidence"],
    color="red", alpha=0.4, label="Positive (bubble)"
)
axes[1].set_ylabel("Confidence")
axes[1].set_ylim(0, 1)
axes[1].axhline(0.5, color="gray", linestyle="--", linewidth=0.5)
axes[1].legend(loc="upper left")
axes[1].set_title("LPPLS Positive Confidence (Bubble Detection)")

# Negative confidence (anti-bubble / capitulation)
axes[2].fill_between(
    conf_df.index, 0, conf_df["negative_confidence"],
    color="green", alpha=0.4, label="Negative (anti-bubble)"
)
axes[2].set_ylabel("Confidence")
axes[2].set_ylim(0, 1)
axes[2].axhline(0.5, color="gray", linestyle="--", linewidth=0.5)
axes[2].legend(loc="upper left")
axes[2].set_title("LPPLS Negative Confidence")

axes[2].xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
plt.tight_layout()
plt.show()

## 5. Use the built-in visualization

In [ ]:
# The fatcrash viz module provides a convenient combined plot
plot_price_with_confidence(
    dates=conf_df.index,
    prices=conf_df["close"].values,
    positive_confidence=conf_df["positive_confidence"].values,
    negative_confidence=conf_df["negative_confidence"].values,
    title="BTC/USD LPPLS Bubble Detection",
)

## 6. Zoom into specific bubble episodes

In [ ]:
episodes = [
    ("2017 Bubble", "2017-06-01", "2018-03-01"),
    ("2021 Bubble", "2021-01-01", "2021-12-01"),
]

fig, axes = plt.subplots(len(episodes), 1, figsize=(14, 5 * len(episodes)))
if len(episodes) == 1:
    axes = [axes]

for ax, (title, start, end) in zip(axes, episodes):
    mask = (conf_df.index >= start) & (conf_df.index <= end)
    subset = conf_df.loc[mask]

    ax2 = ax.twinx()
    ax.plot(subset.index, subset["close"], "k-", linewidth=1, label="Price")
    ax2.fill_between(
        subset.index, 0, subset["positive_confidence"],
        color="red", alpha=0.3, label="LPPLS Confidence"
    )
    ax.set_ylabel("Price (USD)")
    ax2.set_ylabel("Confidence")
    ax2.set_ylim(0, 1)
    ax.set_title(title)
    ax.legend(loc="upper left")
    ax2.legend(loc="upper right")

plt.tight_layout()
plt.show()

## Summary

- The LPPLS model successfully identifies the characteristic **super-exponential growth
  with accelerating oscillations** before major BTC crashes.
- High positive confidence (> 0.5) aligns well with known bubble peaks.
- The confidence indicator is a key input to the aggregated crash signal (notebook 06).
- For a neural-network enhanced version, see notebook 07 (Deep LPPLS).